<a href="https://colab.research.google.com/github/Mohit-Mithra/ceiling-graph/blob/main/TGN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is our implementation of the temporal graph network. The link to the summary of this paper is given here:
https://docs.google.com/document/d/1lrcuWEtM8nBoIRWkQXsAntg0-vmbtxBjrjk8i4ZHtlU/edit?usp=sharing. Complete details of the paper and code will be given there.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!pwd


Mounted at /content/gdrive
/content


In [ ]:
%cd gdrive/My Drive/ceiling-graphs/Phase_2_Setup_entity_graph_TGN/TGN

/content/gdrive/.shortcut-targets-by-id/1M7ehtNpIR8a-P2UBbiz9qn8cEg7ROX3Y/ceiling-graphs/Phase_2_Setup_entity_graph_TGN/TGN


In [ ]:
#!git clone https://github.com/twitter-research/tgn.git

Cloning into 'tgn'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 159 (delta 81), reused 135 (delta 61), pack-reused 0
Receiving objects: 100% (159/159), 971.85 KiB | 2.85 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [ ]:
#%cd tgn/data

[Errno 2] No such file or directory: 'tgn/data'
/content/gdrive/My Drive/Phase_2_Setup_entity_graph_TGN/TGN


In [ ]:
#!wget http://snap.stanford.edu/jodie/reddit.csv

--2020-10-11 04:39:47--  http://snap.stanford.edu/jodie/reddit.csv
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2368348656 (2.2G) [text/csv]
Saving to: ‘reddit.csv’

reddit.csv          100%[===================>]   2.21G  4.85MB/s    in 5m 29s  

2020-10-11 04:45:16 (6.87 MB/s) - ‘reddit.csv’ saved [2368348656/2368348656]



In [ ]:
#!wget http://snap.stanford.edu/jodie/wikipedia.csv

--2020-10-11 04:45:39--  http://snap.stanford.edu/jodie/wikipedia.csv
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 559937473 (534M) [text/csv]
Saving to: ‘wikipedia.csv’

wikipedia.csv       100%[===================>] 534.00M  8.64MB/s    in 2m 25s  

2020-10-11 04:48:04 (3.69 MB/s) - ‘wikipedia.csv’ saved [559937473/559937473]



In [ ]:
%cd tgn

/content/gdrive/.shortcut-targets-by-id/1M7ehtNpIR8a-P2UBbiz9qn8cEg7ROX3Y/ceiling-graphs/Phase_2_Setup_entity_graph_TGN/TGN/tgn


In [ ]:
!python utils/preprocess_data.py --data wikipedia --bipartite

In [ ]:
# !python utils/preprocess_data.py --data reddit --bipartite

In [ ]:
!python train_self_supervised.py --use_memory --prefix tgn-attn --n_runs 1

INFO:root:Namespace(aggregator='last', backprop_every=1, bs=200, data='wikipedia', different_new_nodes=False, drop_out=0.1, dyrep=False, embedding_module='graph_attention', gpu=0, lr=0.0001, memory_dim=172, memory_update_at_end=False, memory_updater='gru', message_dim=100, message_function='identity', n_degree=10, n_epoch=50, n_head=2, n_layer=1, n_runs=1, node_dim=100, patience=5, prefix='tgn-attn', randomize_features=False, time_dim=100, uniform=False, use_destination_embedding_in_message=False, use_memory=True, use_source_embedding_in_message=False)
The dataset has 157474 interactions, involving 9227 different nodes
The training dataset has 81029 interactions, involving 6141 different nodes
The validation dataset has 23621 interactions, involving 3256 different nodes
The test dataset has 23621 interactions, involving 3564 different nodes
The new node validation dataset has 12016 interactions, involving 2120 different nodes
The new node test dataset has 11715 interactions, involving 

In [ ]:
!python train_supervised.py --use_memory --prefix tgn-attn --n_runs 1

INFO:root:Namespace(aggregator='last', backprop_every=1, bs=100, data='wikipedia', different_new_nodes=False, drop_out=0.1, embedding_module='graph_attention', gpu=0, lr=0.0003, memory_dim=172, memory_update_at_end=False, message_dim=100, message_function='identity', n_degree=10, n_epoch=10, n_head=2, n_layer=1, n_neg=1, n_runs=1, new_node=False, node_dim=100, patience=5, prefix='tgn-attn', randomize_features=False, time_dim=100, uniform=False, use_destination_embedding_in_message=False, use_memory=True, use_source_embedding_in_message=False, use_validation=False)
DEBUG:root:Num of training instances: 133853
DEBUG:root:Num of batches per epoch: 1339
INFO:root:Loading saved TGN model
INFO:root:TGN models loaded
INFO:root:Start training node classification task
INFO:root:Epoch 0: train loss: 0.03854482959080599, val auc: 0.8895355450419707, time: 52.88731265068054
INFO:root:Epoch 1: train loss: 0.01034392374890677, val auc: 0.8840978495991857, time: 54.118865966796875
INFO:root:Epoch 2: 

In [ ]:
import numpy as np
data_name = './data/{}.csv'.format('reddit')
u_list, i_list, ts_list, label_list = [], [], [], []
feat_l = []
idx_list = []

i = 0
with open(data_name) as f:

    s = next(f)
    for idx, line in enumerate(f):
        i += 1
        if(i == 10000):
            break
        e = line.strip().split(',')
        #print(e)
        u = int(e[0])
        i = int(e[1])

        ts = float(e[2])
        label = float(e[3])  # int(e[3])

        feat = np.array([float(x) for x in e[4:]])
        #print(u, i, feat)
        u_list.append(u)
        i_list.append(i)
        ts_list.append(ts)
        label_list.append(label)
        idx_list.append(idx)

        feat_l.append(feat)





TypeError: ignored

In [ ]:
while(1):
    i = 0

In [ ]:
print(sum(label_list))

366.0
